Data Mining and Visualisation 2019-2020 <br>
Practical 5 - Time-Series Analysis and Processing - focus on Time-Series Classification<br>
Teaching Assistant Muhammad Usman <br>

This practical requires you to use time series classification approaches to classify multivariate sensor data based on whether the telecommunications network is actually up and running or not. -1 is the negative class denoting downtime and 1 is for optimal network condition. <br>
Use this notebook with CoLab. <br>
You may enable GPU capabilities <br>

You need to undergo a number of steps to fill the gaps below.

In [2]:
# Load all necessary libaries, including keras and tensorflow. 
import keras
import tensorflow as tf
import os
import pandas as pd
import numpy as np

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM

from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


# Load data



In [5]:
from google.colab import files
files.upload() #upload the files needed with this command if you'd like. There are about 20 csv files and one for the target labels

Saving Wireless_Telecom_1.csv to Wireless_Telecom_1.csv
Saving Wireless_Telecom_2.csv to Wireless_Telecom_2.csv
Saving Wireless_Telecom_3.csv to Wireless_Telecom_3.csv
Saving Wireless_Telecom_4.csv to Wireless_Telecom_4.csv
Saving Wireless_Telecom_5.csv to Wireless_Telecom_5.csv
Saving Wireless_Telecom_6.csv to Wireless_Telecom_6.csv
Saving Wireless_Telecom_7.csv to Wireless_Telecom_7.csv
Saving Wireless_Telecom_8.csv to Wireless_Telecom_8.csv
Saving Wireless_Telecom_9.csv to Wireless_Telecom_9.csv
Saving Wireless_Telecom_10.csv to Wireless_Telecom_10.csv
Saving Wireless_Telecom_11.csv to Wireless_Telecom_11.csv
Saving Wireless_Telecom_12.csv to Wireless_Telecom_12.csv
Saving Wireless_Telecom_13.csv to Wireless_Telecom_13.csv
Saving Wireless_Telecom_14.csv to Wireless_Telecom_14.csv
Saving Wireless_Telecom_15.csv to Wireless_Telecom_15.csv
Saving Wireless_Telecom_16.csv to Wireless_Telecom_16.csv
Saving Wireless_Telecom_17.csv to Wireless_Telecom_17.csv
Saving Wireless_Telecom_18.csv t

{'Wireless_Telecom_1.csv': b'Sensor 1,Sensor 2,Sensor 3,Sensor 4\r\n-0.85714,-0.52,0.95238,0.35\r\n-1,-0.76,0.71429,0.35\r\n-1,-0.8,0.85714,0.35\r\n-1,-0.44,0.71429,0.55\r\n-1,-0.32,0.7619,0.65\r\n-1,-0.56,0.95238,0.65\r\n-1,-0.6,0.90476,0.8\r\n-1,-0.72,0.80952,0.9\r\n-0.80952,-0.52,0.47619,0.8\r\n-0.66667,-0.6,0.42857,0.8\r\n-1,-0.28,0.14286,0.5\r\n-0.71429,-0.2,0.095238,0.45\r\n-0.52381,-0.08,0.095238,0.65\r\n-0.47619,-0.44,0.28571,0.55\r\n-0.28571,-0.44,0.095238,0.15\r\n-0.2381,-0.08,-0.19048,-0.15\r\n-0.2381,-0.12,0.14286,0.2\r\n-0.28571,-0.44,-0.14286,0.35\r\n-0.28571,-0.44,-0.095238,0\r\n-0.19048,-0.44,0.19048,0.15\r\n-0.14286,-0.16,-0.14286,0.2\r\n-0.28571,-0.04,0,0.2\r\n-0.095238,0.04,0.14286,-0.4\r\n0,-0.08,0,0.05',
 'Wireless_Telecom_10.csv': b'Sensor 1,Sensor 2,Sensor 3,Sensor 4\r\n-0.57143,-0.36,0.57143,0.8\r\n-0.7619,-0.24,0.42857,0.65\r\n-0.95238,-0.2,0.14286,0.8\r\n-0.85714,-0.16,0.14286,0.55\r\n-0.66667,-0.2,0.38095,0.35\r\n-0.57143,-0.48,0.2381,0.5\r\n-0.47619,-0.4,0.1

In [6]:
!ls

sample_data		 Wireless_Telecom_17.csv  Wireless_Telecom_3.csv
Wireless_Telecom_10.csv  Wireless_Telecom_18.csv  Wireless_Telecom_4.csv
Wireless_Telecom_11.csv  Wireless_Telecom_19.csv  Wireless_Telecom_5.csv
Wireless_Telecom_12.csv  Wireless_Telecom_1.csv   Wireless_Telecom_6.csv
Wireless_Telecom_13.csv  Wireless_Telecom_20.csv  Wireless_Telecom_7.csv
Wireless_Telecom_14.csv  Wireless_Telecom_21.csv  Wireless_Telecom_8.csv
Wireless_Telecom_15.csv  Wireless_Telecom_22.csv  Wireless_Telecom_9.csv
Wireless_Telecom_16.csv  Wireless_Telecom_2.csv   Wireless_Telecom_target.csv


In [0]:
data_files = [f for f in os.listdir() if f.endswith('csv')]

In [8]:
#Read one of the files and printi out with name.head(). Also try to return its shape with name.shape
df = pd.read_csv(data_files[0])
df.head()

,Sensor 1,Sensor 2,Sensor 3,Sensor 4
0,-0.85714,-0.48,0.71429,0.20
1,-0.57143,-0.36,0.52381,0.80
2,-0.71429,-0.24,0.71429,0.80
3,-0.57143,-0.32,0.76190,0.55
4,-0.71429,-0.60,0.71429,0.65


In [9]:
df.shape

(24, 4)

In [10]:
# Load all data files
sequences = []
path = 'Wireless_Telecom'

for i in range(1, len(data_files)):
    file_path = f'{path}_{i}.csv'
    print(file_path)
    df = pd.read_csv(file_path, header=0)
    values = df.values
    sequences.append(values)

Wireless_Telecom_1.csv
Wireless_Telecom_2.csv
Wireless_Telecom_3.csv
Wireless_Telecom_4.csv
Wireless_Telecom_5.csv
Wireless_Telecom_6.csv
Wireless_Telecom_7.csv
Wireless_Telecom_8.csv
Wireless_Telecom_9.csv
Wireless_Telecom_10.csv
Wireless_Telecom_11.csv
Wireless_Telecom_12.csv
Wireless_Telecom_13.csv
Wireless_Telecom_14.csv
Wireless_Telecom_15.csv
Wireless_Telecom_16.csv
Wireless_Telecom_17.csv
Wireless_Telecom_18.csv
Wireless_Telecom_19.csv
Wireless_Telecom_20.csv
Wireless_Telecom_21.csv
Wireless_Telecom_22.csv


In [11]:
targets = pd.read_csv(f'{path}_target.csv').values[:,1]
targets

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1])

In [12]:
# Sequences have different lengths. So we need to make that uniform based on some statistics.
# You need to provide some statistics with regards to the each sequence lengths, i.e. some sequences are longer than others.
# use .describe() to get an idea of what the spread of the sequence lengths is
len_sequences = [len(s) for s in sequences]
pd.Series(len_sequences).describe()

count    22.000000
mean     25.818182
std       2.152216
min      23.000000
25%      24.000000
50%      25.500000
75%      27.000000
max      30.000000
dtype: float64

In [13]:
print(len_sequences)

[24, 24, 24, 24, 24, 24, 23, 27, 28, 24, 26, 29, 29, 25, 30, 27, 29, 23, 25, 27, 26, 26]


In [14]:
#add padding to the max length based on the value you got from .describe()
to_pad = max(len_sequences)
new_seq = []
for i, seq in enumerate(sequences):
    n = to_pad - len_sequences[i]
    to_concat = np.repeat(seq[-1], n).reshape(4, n).T
    new_seq.append(np.concatenate([seq, to_concat]))
current_seq = np.stack(new_seq)
current_seq.shape

(22, 30, 4)

In [15]:
#truncate the sequences to having 24 values
from keras.preprocessing import sequence
seq_len = 24
final_seq = current_seq[:, :24, :]
final_seq.shape

(22, 24, 4)

In [0]:
# now create train and test sets.....
# for the sake of this example we will consider the test set as being both a test and validation set,
# but you can just adjust it accordingly if you want to create a validation set as well
split = 17

train = final_seq[:split]
test = final_seq[split:]

train_target = targets[:split]
test_target = targets[split:]

In [17]:
train_target = (train_target+1)/2
test_target = (test_target+1)/2
train_target

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0.])

# LSTM

In [18]:
# create an LSTM model having 512 units. It is a binary problem so you can use a dense layer with value 1 and activation sigmoid
model = Sequential()
model.add(LSTM(256, input_shape=(seq_len, 4)))
model.add(Dense(1, activation='sigmoid'))
model

In [19]:
#now you need to actually define the training process for the LSTM by adding optimiser (use ADAM or any other you want) and also play with different values for learning rate
# create a check point so that you can get an outcome for each epoch

adam = Adam(lr=0.001)
chk = ModelCheckpoint('best_model.pkl', monitor='val_acc', save_best_only=True, mode='max', verbose=1)
model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
model.fit(train, train_target, epochs=200, batch_size=128, callbacks=[chk], validation_data=(test,test_target))

Epoch 1/200
1/1 [==============================] - 0s 404ms/step - loss: 0.7095 - accuracy: 0.3529 - val_loss: 0.6730 - val_accuracy: 1.0000
Epoch 2/200
1/1 [==============================] - 0s 33ms/step - loss: 0.6687 - accuracy: 0.9412 - val_loss: 0.7343 - val_accuracy: 0.0000e+00
Epoch 3/200
1/1 [==============================] - 0s 37ms/step - loss: 0.6346 - accuracy: 0.5882 - val_loss: 0.8099 - val_accuracy: 0.0000e+00
Epoch 4/200
1/1 [==============================] - 0s 33ms/step - loss: 0.6013 - accuracy: 0.5882 - val_loss: 0.9191 - val_accuracy: 0.0000e+00
Epoch 5/200
1/1 [==============================] - 0s 29ms/step - loss: 0.5665 - accuracy: 0.6471 - val_loss: 1.0814 - val_accuracy: 0.0000e+00
Epoch 6/200
1/1 [==============================] - 0s 29ms/step - loss: 0.5344 - accuracy: 0.6471 - val_loss: 1.1840 - val_accuracy: 0.0000e+00
Epoch 7/200
1/1 [==============================] - 0s 28ms/step - loss: 0.5009 - accuracy: 0.5882 - val_loss: 1.0163 - val_accuracy: 0.0000

In [21]:
!ls


sample_data		 Wireless_Telecom_17.csv  Wireless_Telecom_3.csv
Wireless_Telecom_10.csv  Wireless_Telecom_18.csv  Wireless_Telecom_4.csv
Wireless_Telecom_11.csv  Wireless_Telecom_19.csv  Wireless_Telecom_5.csv
Wireless_Telecom_12.csv  Wireless_Telecom_1.csv   Wireless_Telecom_6.csv
Wireless_Telecom_13.csv  Wireless_Telecom_20.csv  Wireless_Telecom_7.csv
Wireless_Telecom_14.csv  Wireless_Telecom_21.csv  Wireless_Telecom_8.csv
Wireless_Telecom_15.csv  Wireless_Telecom_22.csv  Wireless_Telecom_9.csv
Wireless_Telecom_16.csv  Wireless_Telecom_2.csv   Wireless_Telecom_target.csv


In [22]:
chk

In [24]:
#load the model created in order to estimate the accuracy on the training set
#use the model.predict_classes to get the dataset you need to make the predictions and then compare the test_predicted values against the test_target one to get the final accuracy score

#loading the model and checking accuracy on the test data
# model = load_model('best_model.pkl')

model = chk.model

from sklearn.metrics import accuracy_score
test_preds = model.predict_classes(test)
accuracy_score(test_target, test_preds)

##end of exercise###

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


1.0